Initiliaze and prepare all models

In [1]:
import matplotlib.pyplot as plt

In [2]:
# ~~~~~~~~~~~~~~~~
# PyTorch Model

import torch
import yaml
from models.decoder import architectures

with open("models/decoder/decoder_params.yaml") as file:
    params = yaml.safe_load(file)

model = architectures.LightningNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
model.load_state_dict(torch.load("models/decoder/decoder.pth"))
model.eval();

model_denoise = architectures.LightningNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
model_denoise.load_state_dict(torch.load("models/decoder/decoder_denoise.pth"))
model_denoise.eval();

# ~~~~~~~~~~~~~~~~
# Kernel DM+V

from models.kernel_dmv.my_kernel_dmv import KernelDMV
kdm = KernelDMV()

# ~~~~~~~~~~~~~~~~
# GMRF

from models.gmrf.my_gmrf import myGMRF
gmrf = myGMRF()

/home/nwinkler/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import scipy.interpolate as interpolate
import numpy as np

def interpolate_batch(X, dropout_probability=0.5):
    """Interpolate a batch of size [batch, channel, width, height]"""
    all_interpolated = torch.empty([1,6,5])
    
    for sample in range(X.shape[0]):
        this_X = X[sample][0][0].clone()
        
        r = np.linspace(0, 1, this_X.shape[0])
        c = np.linspace(0, 1, this_X.shape[1])

        rr, cc = np.meshgrid(c, r)
        mask = this_X.bernoulli(1-dropout_probability).bool()
        
        try:
            f = interpolate.Rbf(rr[mask], cc[mask], this_X[mask], function='linear')
            interpolated = torch.tensor(f(rr, cc)).unsqueeze(0).float()
        except:
            interpolated = this_X   

        all_interpolated = torch.cat([all_interpolated, interpolated])
        
    all_interpolated = all_interpolated.unsqueeze(1).unsqueeze(1)
    return all_interpolated[1:]

### Data & Metrics

Specify data

In [22]:
from torch.utils import data
from data.gdm_dataset import GasDataSet

dataset = GasDataSet("data/30x25/test.pt")

### Run

In [5]:
list_of_models = [
                  "decoder",
                  "gmrf",
                  "kdm",
                 ]

In [79]:
from tqdm import tqdm
import gdm_metrics

def run_test(dropout_probability):
    rmse = {"decoder": 0,
            "gmrf": 0,
            "kdm": 0}

    kld = {"decoder": 0,
           "gmrf": 0,
           "kdm": 0}
    
    dataloader = iter(data.DataLoader(dataset, shuffle=False, drop_last=True))

    for X, y in tqdm(dataloader):
        X_drop = interpolate_batch(X, dropout_probability)
        X_drop = torch.clamp(X_drop, min=0)

        with torch.no_grad(): 
            if "decoder" in list_of_models:
                y_decoder = model(X_drop.squeeze(1))
                rmse["decoder"] += gdm_metrics.rmse(y_decoder, y)
                kld["decoder"] += gdm_metrics.kld(y_decoder, y)

            if "gmrf" in list_of_models:
                y_gmrf = gmrf.calculate(X_drop.squeeze())[None,None,:]  # add two empty dimensions to be consistent with y.shape
                rmse["gmrf"] += gdm_metrics.rmse(y_gmrf, y)
                kld["gmrf"] += gdm_metrics.kld(y_gmrf, y)

            if "kdm" in list_of_models:
                y_kdm = kdm.calculate(X_drop.squeeze())[None,None,:]    # add two empty dimensions to be consistent with y.shape
                rmse["kdm"] += gdm_metrics.rmse(y_kdm, y)
                kld["kdm"] += gdm_metrics.kld(y_kdm, y)
                
    return rmse,kld

### Evaluation

Dropout probability = 0

In [7]:
rmse, kld = run_test(0)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [28:41<00:00, 18.82it/s]

########
# RMSE #
########
decoder: 	 0.07058026641607285
gmrf: 	 0.11100773428192476
kdm: 	 0.11176697431124033
########
# KL D #
########
decoder: 	 0.0032490077428519726
gmrf: 	 0.008397889089196814
kdm: 	 0.008162776568291319


Dropout probability = 0.1

In [80]:
rmse, kld = run_test(0.1)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [29:10<00:00, 18.51it/s]

########
# RMSE #
########
decoder: 	 0.08487005531787872
gmrf: 	 0.11642053263419509
kdm: 	 0.11703312566815562
########
# KL D #
########
decoder: 	 0.004725067876279354
gmrf: 	 0.00920623369479877
kdm: 	 0.008985643012736506


Dropout probability = 0.2

In [81]:
rmse, kld = run_test(0.2)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [29:09<00:00, 18.52it/s]

########
# RMSE #
########
decoder: 	 0.09923622012138367
gmrf: 	 0.12260285959104737
kdm: 	 0.12302103115679401
########
# KL D #
########
decoder: 	 0.006354163400828838
gmrf: 	 0.01016387159404957
kdm: 	 0.00995567692282639


Dropout probability = 0.3

In [82]:
rmse, kld = run_test(0.3)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [31:44<00:00, 17.01it/s]

########
# RMSE #
########
decoder: 	 0.11317837238311768
gmrf: 	 0.12925757799412382
kdm: 	 0.12943918194590892
########
# KL D #
########
decoder: 	 0.00807542260736227
gmrf: 	 0.011217099381724478
kdm: 	 0.011019703035804879


Dropout probability = 0.4

In [83]:
rmse, kld = run_test(0.4)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [31:21<00:00, 17.22it/s]

########
# RMSE #
########
decoder: 	 0.12853094935417175
gmrf: 	 0.1374386598504956
kdm: 	 0.1372600537708818
########
# KL D #
########
decoder: 	 0.010160161182284355
gmrf: 	 0.012562012110366979
kdm: 	 0.012366408932381097


Dropout probability = 0.5

In [84]:
rmse, kld = run_test(0.5)

print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataset)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataset)}")

100%|█████████████████████████████████████████████████████████████████████| 32400/32400 [31:39<00:00, 17.06it/s]

########
# RMSE #
########
decoder: 	 0.1441357433795929
gmrf: 	 0.1467790664119138
kdm: 	 0.1461100295940834
########
# KL D #
########
decoder: 	 0.012422662228345871
gmrf: 	 0.014128353294522823
kdm: 	 0.013925408417334011
